In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("conll2003")

In [ ]:
print("Доступные сплиты:", dataset)

Доступные сплиты: DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [ ]:
train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

print("\nПример данных:")
print(train_data[0])


Пример данных:
{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [ ]:
print("\nКолонки в датасете:", train_data.column_names)


Колонки в датасете: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags']


In [ ]:
ner_tags = train_data.features['ner_tags'].feature.names
print("\nСущности в разметке (NER-теги):", ner_tags)


Сущности в разметке (NER-теги): ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [ ]:
!pip install sklearn-crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.5 MB/s eta 0:00:00


In [ ]:
# Функция для извлечения признаков из токена
def extract_features(tokens, pos_tags, chunk_tags, i):
    token = tokens[i]
    # Признаки самого токена
    features = {
        'bias': 1.0,  # bias feature (всегда равен 1 для всех токенов)
        'token.lower()': token.lower(),  # Токен в нижнем регистре
        'token.isupper()': token.isupper(),  # Весь токен в верхнем регистре?
        'token.istitle()': token.istitle(),  # Начинается ли токен с заглавной буквы?
        'token.isdigit()': token.isdigit(),  # Содержит ли токен цифры?
        'pos_tag': pos_tags[i],  # Часть речи (POS-тег)
        'chunk_tag': chunk_tags[i],  # Chunk-тег
    }

    # Признаки для соседних слов (контекст)
    if i > 0:
        prev_token = tokens[i-1]
        features.update({
            '-1:token.lower()': prev_token.lower(),
            '-1:token.isupper()': prev_token.isupper(),
            '-1:token.istitle()': prev_token.istitle(),
            'pos_tag-1': pos_tags[i-1],
            'chunk_tag-1': chunk_tags[i-1],
        })
    else:
        features['BOS'] = True  # Признак для начала предложения

    if i < len(tokens)-1:
        next_token = tokens[i+1]
        features.update({
            '+1:token.lower()': next_token.lower(),
            '+1:token.isupper()': next_token.isupper(),
            '+1:token.istitle()': next_token.istitle(),
            'pos_tag+1': pos_tags[i+1],
            'chunk_tag+1': chunk_tags[i+1],
        })
    else:
        features['EOS'] = True  # Признак для конца предложения

    return features

# Преобразуем данные в нужный формат для CRF
def prepare_data(data):
    sentences = []
    labels = []
    for entry in data:
        tokens = entry['tokens']
        pos_tags = entry['pos_tags']
        chunk_tags = entry['chunk_tags']
        ner_tags = entry['ner_tags']

        sentence_features = [extract_features(tokens, pos_tags, chunk_tags, i) for i in range(len(tokens))]
        sentences.append(sentence_features)
        labels.append([ner_tags[i] for i in range(len(tokens))])

    return sentences, labels

In [ ]:
# Преобразование всех сплитов в признаки и метки
X_train, y_train = prepare_data(train_data)
X_val, y_val = prepare_data(val_data)
X_test, y_test = prepare_data(test_data)

# Функция для преобразования числовых меток в текстовые
def convert_labels(labels):
    return [[ner_tags[label] for label in sentence] for sentence in labels]

# Преобразуем тренировочные, валидационные и тестовые метки из чисел в текстовые представления
y_train = convert_labels(y_train)
y_val = convert_labels(y_val)
y_test = convert_labels(y_test)

# Пример того, как выглядят признаки для одного предложения
print("Пример признаков для одного токена в предложении:")
print(X_train[0][0])

Пример признаков для одного токена в предложении:
{'bias': 1.0, 'token.lower()': 'eu', 'token.isupper()': True, 'token.istitle()': False, 'token.isdigit()': False, 'pos_tag': 22, 'chunk_tag': 11, 'BOS': True, '+1:token.lower()': 'rejects', '+1:token.isupper()': False, '+1:token.istitle()': False, 'pos_tag+1': 42, 'chunk_tag+1': 21}


In [ ]:
# Создание и настройка модели CRF
import sklearn_crfsuite
from sklearn_crfsuite import metrics
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

# Обучаем модель на тренировочном наборе
crf.fit(X_train, y_train)

# Предсказания на валидационном наборе
y_val_pred = crf.predict(X_val)

# Оценка на валидационном наборе
labels = list(crf.classes_)
labels.remove('O')  # Убираем тег 'O', чтобы он не доминировал в метриках

# Выводим метрики на валидационном наборе
print("Оценка на валидационном наборе:")
print(metrics.flat_classification_report(y_val, y_val_pred, labels=labels))

# Предсказания на тестовом наборе
y_test_pred = crf.predict(X_test)

# Оценка на тестовом наборе
print("Оценка на тестовом наборе:")
print(metrics.flat_classification_report(y_test, y_test_pred, labels=labels))

Оценка на валидационном наборе:
              precision    recall  f1-score   support

       B-ORG       0.88      0.82      0.85      1341
      B-MISC       0.92      0.82      0.87       922
       B-PER       0.91      0.88      0.90      1842
       I-PER       0.94      0.94      0.94      1307
       B-LOC       0.91      0.89      0.90      1837
       I-ORG       0.82      0.82      0.82       751
      I-MISC       0.87      0.73      0.79       346
       I-LOC       0.94      0.75      0.84       257

   micro avg       0.90      0.86      0.88      8603
   macro avg       0.90      0.83      0.86      8603
weighted avg       0.90      0.86      0.88      8603

Оценка на тестовом наборе:
              precision    recall  f1-score   support

       B-ORG       0.82      0.69      0.75      1661
      B-MISC       0.80      0.74      0.77       702
       B-PER       0.84      0.85      0.84      1617
       I-PER       0.85      0.95      0.90      1156
       B-LOC       

In [ ]:
#Проверим работу модели на примере:

In [ ]:
# Пример предложения для предсказания
example_sentence = ["John", "Smith", "is", "from", "New", "York", "and", "works", "at", "Google", "."]

# POS-теги и chunk-теги для примера (можно взять случайные для демонстрации)
# В реальных случаях это должны быть теги, предсказанные другой моделью или подготовленные вручную
example_pos_tags = [22, 22, 42, 35, 16, 16, 35, 42, 35, 16, 7]  # Это пример
example_chunk_tags = [11, 11, 21, 21, 11, 12, 21, 22, 21, 11, 0]  # Это пример

# Преобразуем предложение в признаки для CRF
example_features = [extract_features(example_sentence, example_pos_tags, example_chunk_tags, i) for i in range(len(example_sentence))]

# Делаем предсказание
predicted_labels = crf.predict([example_features])[0]  # Модель предсказывает текстовые метки (например, B-PER, O и т.д.)

# Выводим результаты
for token, label in zip(example_sentence, predicted_labels):
    print(f"{token}: {label}")

John: B-PER
Smith: I-PER
is: O
from: O
New: B-LOC
York: I-LOC
and: O
works: O
at: O
Google: B-LOC
.: O
